In [22]:
import math
import random
import numpy as np
from scipy.stats import poisson
from scipy.stats import norm
n = 7
ch = 1
cp = 10
co = 320
cr = 20
max_inv_level = 250
min_inv_level =180
means=[-1]*n
__opt_cost = []
expected_cost = 0
print('good')

good


In [23]:
for i in range(n):
    means[i] = round(50 * (1 + math.sin(math.pi * i /6))) + min(i, n-i)
init_inv = 0
threshold = 0.0001
print('good')

good


In [24]:
max_demand = int(poisson.ppf(loc = 0 , mu = max(means), q = (1-threshold)))
prob = [[0.0 for _ in range(0, max_demand +1)] for _ in range(n)]
for i in range(n):
    for j in range(max_demand +1):
        if means[i] != 0 :
            prob[i][j] = poisson.pmf(k = j, mu = means[i])
        else:
            prob[i][j] = poisson.pmf(k = j, mu = 0.1)  # 这些都和上面的过程很像
    prob[i] = prob[i]/sum(prob[i])


demand = [0 for _ in range(n * max_demand + 1)]  # n个时期，最多可以累积n * max_demand的需求
for i in range(max_demand + 1):
    demand[i] = prob[0][i]
for i in range(1, n):
    temp = [0 for _ in range(n * max_demand + 1)]
    for a in range((n - 1) * max_demand + 1):
        for b in range(max_demand + 1):
            temp[a + b] +=prob[i][b] * demand[a]  # 求出需求为a+b的概率
    demand = temp
j = n * max_demand
pdf_dem = demand[j]
while pdf_dem < threshold:  # 从后往前 将发生概率比较小的需求视为不可能事件
    j -= 1
    pdf_dem += demand[j - 1]

max_inv_level = j # 将库存的最大最小值设为可能发生的需求量
min_inv_level = -j + init_inv
print('good')

good


In [25]:
demand = [0 for _ in range(n * max_demand + 1)]  # n个时期，最多可以累积n * max_demand的需求
for i in range(max_demand + 1):
    demand[i] = prob[0][i]
for i in range(1, n):
    temp = [0 for _ in range(n * max_demand + 1)]
    for a in range((n - 1) * max_demand + 1):
        for b in range(max_demand + 1):
            temp[a + b] += prob[i][b] * demand[a]  # 求出需求为a+b的概率
    demand = temp
j = n * max_demand
pdf_dem = demand[j]
while pdf_dem < threshold:  # 从后往前 将发生概率比较小的需求视为不可能事件
    j -= 1
    pdf_dem += demand[j - 1]
    
print('good')

good


In [26]:
conv_prob = np.zeros((n, n, n * max_demand + 1))  # 创造三维数组

for i in range(n):
    for j in range(max_demand+1):
        conv_prob[i][i][j] = prob[i][j]
    # self.conv_prob[i][i] = self.conv_prob[i][i] / sum(self.conv_prob[i][i])

for i in range(0, n):
    for i2 in range(i+1, n):
        for j in range((i2-i)*max_demand +1):
            for j2 in range(max_demand+1):
                conv_prob[i][i2][j+j2] += conv_prob[i][i2-1][j] * prob[i2][j2]
        conv_prob[i][i2] = conv_prob[i][i2] / sum(conv_prob[i][i2])
        
print('good')

good


In [27]:
__opt_cost = [[float("inf") for _ in range(min_inv_level, max_inv_level + 1)] for _ in range(n + 1)]
opt_q = [[max_inv_level for _ in range(min_inv_level, max_inv_level +1)] for _ in range(n+1)]
opt_r = [[0 for _ in range(min_inv_level, max_inv_level +1)] for _ in range(n+1)]     

for i in range(min_inv_level, max_inv_level +1):
    __opt_cost[n][i] = 0
    
print('good')    

good


In [61]:
def cost(t, i, r):
    if t>n-1:
        return
    if t+r-1>n-1:
        return
    if r=0:
        return
    temp = 0.0
    for d, p in enumerate(conv_prob[t][t + r - 1]):
        close_inv = i - d
        if close_inv <= min_inv_level:
            temp = temp + p * cost(t+r-1, min_inv_level, n-t-r)
        else:
            temp = temp + p * cost(t+r-1, close_inv, n-t-r)
    for j in range(r):
        for d, p in enumerate(conv_prob[t][t + j]):
            close_inv = i - d
            if close_inv >= 0:
                temp = temp + p * ch * close_inv
            if close_inv < 0:
                temp = temp + p * (-cp) * close_inv
    return temp    
print('good')

good


In [56]:
min_inv_level=0
max_inv_level=20
print(min_inv_level)
print(max_inv_level)

0
20


In [62]:
t=0
i=0
bsf_cost = float("inf")
bsf_q = float("inf")
bsf_r = float("inf")
s = [float("inf")] * n
S = [float("inf")] * n
R = [0] * n
q_limit = max_inv_level - i
for r in range(1, n - t+1):
    for q in range(0, q_limit + 1):
        temp = cr if q == 0 else cr+co
        temp += cost(t,i+q,r)
        if temp < bsf_cost:
            bsf_cost = temp
            bsf_q = q
            bsf_r = r
t=t+bsf_r
R[t-1]=1
while t<n:
    for i in range(0, max_inv_level + 1):
        for r in range(1, n - t+1):
            for q in range(0, q_limit + 1):
                temp = cr if q == 0 else cr+co
                temp += cost(t,i+q,r)
                if temp < bsf_cost:
                    bsf_cost = temp
                    bsf_q = q
                    bsf_r = r
                    bsf_i = i
        if i==max_inv_level:
            break            
    t=t+bsf_r
    R[t-1]=1
    s[t]=bsf_i
    S[t]=bsf_i+bsf_q
print('good')

TypeError: unsupported operand type(s) for +=: 'int' and 'NoneType'

In [ ]:
print("Reviews:\t" + str(R))
print("s:\t\t\t" + str(s))
print("S:\t\t\t" + str(S))